In [53]:
# -*- coding: utf-8 -*-
import gzip
import pickle
from urllib import request
import plotly as py
import plotly.express as px
from matplotlib import pyplot
import copy
import torch
import numpy as np

pyplt = py.offline.plot

filename = [
    ["training_images", "train-images-idx3-ubyte.gz"],
    ["test_images", "t10k-images-idx3-ubyte.gz"],
    ["training_labels", "train-labels-idx1-ubyte.gz"],
    ["test_labels", "t10k-labels-idx1-ubyte.gz"]
]


def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading " + name[1] + "...")
        request.urlretrieve(base_url + name[1], name[1])  # url和filename
    print("Download complete.")


def save_mnist():
    mnist = {}

    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,
                                                                                  28 * 28)  # 前面16个字节是magic number, number of images, rows, columns
            print(name[0], mnist[name[0]].shape)

    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)  # 前面8个字节是magic number, number of items
            print(name[0], mnist[name[0]].shape)

    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist, f)
    print("Save complete.")


def load():
    with open("mnist.pkl", 'rb') as f:
        mnist = pickle.load(f)
    image_0 = copy.deepcopy(mnist["training_images"][0])
    image_0 = image_0.reshape(28, 28)
    # px.imshow(image_0).show()
    # pyplt(px.imshow(image_0))

    # pyplot.imshow(image_0.reshape(28, 28))
    # px.imshow(image_0.reshape(28, 28)).show()
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

# download_mnist()
# save_mnist()
# load()

In [54]:
x_train, y_train, x_valid, y_valid = map(torch.tensor, load())
n, c = x_train.shape
x_train, y_train, x_valid, y_valid

(tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8),
 tensor([5, 0, 4,  ..., 5, 6, 8], dtype=torch.uint8),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8),
 tensor([7, 2, 1,  ..., 4, 5, 6], dtype=torch.uint8))

In [55]:
from torch import nn
import torch.nn.functional as fu


class Mnist_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, 128)
        self.hidden2 = nn.Linear(128, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        x = fu.relu(self.hidden1(x))
        x = fu.relu(self.hidden2(x))
        x = self.out(x)
        return x


net = Mnist_NN()
print(net)

Mnist_NN(
  (hidden1): Linear(in_features=784, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)


In [56]:
for name, parameter in net.named_parameters():
    print(name, parameter, parameter.size())

hidden1.weight Parameter containing:
tensor([[ 0.0186, -0.0263,  0.0310,  ..., -0.0343,  0.0238, -0.0073],
        [ 0.0173,  0.0322,  0.0310,  ..., -0.0317, -0.0203, -0.0261],
        [-0.0140,  0.0101, -0.0258,  ...,  0.0238,  0.0324,  0.0331],
        ...,
        [-0.0333,  0.0070,  0.0133,  ...,  0.0126, -0.0298,  0.0262],
        [-0.0199, -0.0126, -0.0169,  ...,  0.0268,  0.0103,  0.0050],
        [ 0.0177,  0.0313, -0.0104,  ...,  0.0033, -0.0275, -0.0312]],
       requires_grad=True) torch.Size([128, 784])
hidden1.bias Parameter containing:
tensor([ 0.0353, -0.0171, -0.0354,  0.0146,  0.0017,  0.0109,  0.0227,  0.0042,
         0.0133,  0.0322, -0.0209,  0.0025,  0.0114,  0.0336,  0.0262,  0.0275,
        -0.0223, -0.0135, -0.0096, -0.0060, -0.0293, -0.0237, -0.0100,  0.0219,
        -0.0327, -0.0241,  0.0294,  0.0258,  0.0287,  0.0197, -0.0045, -0.0155,
         0.0167, -0.0011,  0.0281, -0.0001,  0.0347, -0.0193, -0.0326, -0.0104,
        -0.0056,  0.0195, -0.0196,  0.0129, 

In [94]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim

train_ds = TensorDataset(x_train.float(), y_train.long())
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)

valid_ds = TensorDataset(x_valid.float(), y_valid.long())
valid_dl = DataLoader(valid_ds, batch_size=64 * 2)


def loss_batch(model, loss_func, xb, yb, opt=None):
    pred = model(xb)
    # print(pred.size(), yb.size())
    loss = loss_func(pred, yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


def fit(steps, model, loss_func, opt, train_dl, valid_dl):
    for step in range(steps):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print('当前step:' + str(step), '验证集损失：' + str(val_loss))


def get_model():
    model = Mnist_NN()
    return model, optim.SGD(model.parameters(), lr=0.001)



In [95]:
model, opt = get_model()
loss_func = fu.cross_entropy
fit(25, model, loss_func, opt, train_dl, valid_dl)

当前step:0 验证集损失：0.2523824237704277
当前step:1 验证集损失：0.1875616130888462
当前step:2 验证集损失：0.16522809041142464
当前step:3 验证集损失：0.13846269603073597
当前step:4 验证集损失：0.13073131066709756
当前step:5 验证集损失：0.12863761966526507
当前step:6 验证集损失：0.12087992445230485
当前step:7 验证集损失：0.12133241846896708
当前step:8 验证集损失：0.11699634975865483
当前step:9 验证集损失：0.12291003449447453
当前step:10 验证集损失：0.11526939202789217
当前step:11 验证集损失：0.11324118216121569
当前step:12 验证集损失：0.11018881581686438
当前step:13 验证集损失：0.11262585328249261
当前step:14 验证集损失：0.10969992533992044
当前step:15 验证集损失：0.1121695591959171
当前step:16 验证集损失：0.10989524005297571
当前step:17 验证集损失：0.10853163494067267
当前step:18 验证集损失：0.10913926109597087
当前step:19 验证集损失：0.10986094534662552
当前step:20 验证集损失：0.10964492690949701
当前step:21 验证集损失：0.10900275349356234
当前step:22 验证集损失：0.11207148272308987
当前step:23 验证集损失：0.10951954120901645
当前step:24 验证集损失：0.1107479081864818


In [61]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [4, 5, 6, 7, 8]
list(zip(a, b))

[(1, 4), (2, 5), (3, 6)]

In [82]:
for xb, yb in train_dl:
    print(xb.size(), yb.size(), xb.dtype, yb.dtype)
for xb, yb in valid_dl:
    print(xb.size(), yb.size(), xb.dtype, yb.dtype)

torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) torch.Size([64]) torch.float32 torch.int64
torch.Size([64, 784]) tor

In [96]:
def func1(a1, b1):
    return a1 + 1, b1 + 2


vec = [(0, 1), (2, 3), (4, 5)]
p1, p2 = zip(*[func1(xa, ya) for xa, ya in vec])
print(p1)
print(p2)
print([func1(xa, ya) for xa, ya in vec])

(1, 3, 5)
(3, 5, 7)
[(1, 3), (3, 5), (5, 7)]
